# Analyzing time series data for a given gridding 

- provide path for attributed_voronoi_all_years.geojson and tesselation_points.geojson
- provide output path for the results


In [2]:
# Import relevant packages
import numpy as np
import pandas as pd
import random
import PIL
import rasterio
import shapefile
import geopandas as gpd
import matplotlib.pyplot as plt
from PIL import Image
from pyproj import Transformer
from rasterio import features
from rasterio.enums import Resampling
from rasterio.mask import mask, raster_geometry_mask
from rasterio.plot import show
from shapely.geometry import Polygon, MultiPolygon, mapping
from skimage.measure import block_reduce
from matplotlib.collections import PatchCollection
from matplotlib.colors import Normalize
from matplotlib.colors import LogNorm
from bridson import poisson_disc_samples
import matplotlib.patches as patches
import scienceplots 
import os 
import Tesselation as tess
import inference as infce
import ipykernel
import json

os.environ['JAVA_HOME'] = '/opt/homebrew/opt/openjdk/libexec/openjdk.jdk/Contents/Home'
os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['PATH']

import igraph as ig


In [5]:
def analyze_network(geojson_path, output_path):
    data_array = infce.create_dataframe_from_voronoi(geojson_path)
    te_matrix, p_value_matrix = infce.compute_TE_significance(data_array)
    te_matrix = infce.correct_TE_matrix(te_matrix, p_value_matrix, save=False)
    attributes = infce.measure_attributes_of_graph(te_matrix)

    geojson_path = '/Users/mengeshi/Documents/GitHub/eScience/Data_summary/Tesselations/tessellation_points_1.geojson'
    geojson_data = json.load(open(geojson_path))
    new_geojson_data = infce.assign_attributes_to_geojson(attributes, geojson_data=geojson_data)
    #save new data file using .tofile
    # Convert updated GeoJSON to GeoDataFrame
    gdf = gpd.GeoDataFrame.from_features(new_geojson_data['features'])  
    gdf.set_crs(epsg=4326, inplace=True)
    # Save the new GeoDataFrame to a file
    gdf.to_file(output_path, driver='GeoJSON')


In [6]:

geojson_path = '../Data_summary/Tesselations/attributed_voronoi_all_years_1.geojson'
output_path = '../Data_summary/Tesselations/new_attributed_points.geojson'

analyze_network(geojson_path, output_path)